# 01: General `lumispy` workflow
## Energy conversion and model fitting

This notebook shows:

- Plotting cathodoluminescence data in different ways
- Converting wavelength signal axis to energy signal axis
(**If `non-uniform-axes` hyperspy branch is installed.**)
- Gaussian fitting

Import packages:

In [2]:
%matplotlib qt
#For pop-up window plots, with interactive functionality. If error, use instead %matplotlib tk 
import lumispy as lum
import os
import matplotlib.pyplot as plt

## Loading the pre-processed CL-SEM file

Load the `.hspy` file, which has already been pre-processed (background subtracted, spike removed and spectral corrected)

*You can also leave the path empty. A pop-up window will appear to select the `.hspy` file from the browser.*

In [3]:
# Load file
path = os.path.relpath("demo-files/01/01_demo.hspy")
file = lum.load(path)
axes = (file.axes_manager.navigation_axes[1].get_axis_dictionary(),
        file.axes_manager.navigation_axes[0].get_axis_dictionary(),
        file.axes_manager.signal_axes[0].get_axis_dictionary()
        )

# Create the CL_SEM object with the file data and metadata
cl_sem = lum.signals.cl_sem_spectrum.CLSEMSpectrum(file, axes=axes)
cl_sem.metadata = file.metadata
print(cl_sem)

<CLSEMSpectrum, title: cathodoluminescence_example, dimensions: (40, 30|1015)>


In [4]:
cl_sem.metadata

├── Acquisition_instrument
│   ├── CCD
│   │   ├── amplification = 1
│   │   ├── binning = 1
│   │   ├── channels = 1024
│   │   ├── exposure_time_s = 0.05
│   │   └── readout_rate_khz = 1
│   ├── SEM
│   │   ├── FOV = 8356.175781
│   │   ├── aperture_um = 100.0
│   │   ├── beam_acc_voltage_kv = 6.003
│   │   ├── chamber_pressure_torr = 1.1101e-07
│   │   ├── dwell_time_scan_s = 0.050000000999999995
│   │   ├── gun_lens_amps = 1.2
│   │   ├── obj_lens_amps = 0.326228
│   │   ├── real_magnification = 8356.1757
│   │   ├── resolution_x = 256
│   │   └── resolution_y = 256
│   ├── Spectrometer
│   │   ├── central_wavelength_nm = 500.0
│   │   └── grating = 150.0
│   └── acquisition_system = cambridge_uk_attolight
├── General
│   ├── folder_path = C:\Users\jf631\Documents\GitHub\jordiferrero\nanoDTC\python_demo_notebooks\luminescence_demos\data\cathodoluminescence_example
│   └── title = cathodoluminescence_example
└── Signal
    ├── background = array([[224.11201, 224.65461, 225.1972 , ..., 771.68262, 772.21087,
        77 ... 4.     , 654.     , 654.     , ..., 653.     , 654.     ,
        654.     ]])
    ├── binned = False
    ├── cosmic_rays_subtracted_extra_percent = 50
    ├── cropped_edges = 5
    ├── grating_corrected = True
    └── signal_type = CL_SEM

## Plotting data

Plot the hyperspectral data:

In [5]:
cl_sem.plot()

Plot the average CL spectrum:

In [6]:
cl_sem.mean().plot()

### Chromatic imaging:

Plot the panchromatic image:

In [7]:
cl_sem.T.mean().plot(cmap='viridis')

Get the colour filtered panchromatic images.
Select the energy region to plot as coloured image.

In [8]:
import hyperspy.api as hs

im = cl_sem.T
im.plot()
roi1 = hs.roi.SpanROI(left=452, right=634)      #sets a digitalbandfilter
im_roi1 = roi1.interactive(im, color="green")

In [9]:
im_filtered = roi1(im).mean()

roi_width = roi1.right - roi1.left
roi_centre = roi1.left + 0.5* roi_width

im_filtered.metadata.General.title = "Colour filtered image of {:.0f} $\pm$ {:.0f} nm".format(roi_centre, roi_width)
im_filtered.plot(cmap='viridis')

## Wavelength to energy conversion

If you have installed the `non-uniform-axes` development hyperspy version, this will work:

In [11]:
cl_sem_eV = cl_sem.to_eV(inplace=False)
cl_sem_eV.plot()

NotImplementedError: Conversion to energy axis works only if the non_uniform_axis branch of hyperspy is used.

## Fitting Gaussian

Select the starting parameters

In [12]:
cl_sem.plot()

In [13]:
####################################
#MODEL
g1_centre = 513   # Guess for centre wavelength
g1_max = g1_centre + 20      # Max value for centre wavelength
g1_min = g1_centre - 20      # Min value for centre wavelength

g1_fwhm = 50            #Guess for FWHM
g1_fwhm_max = 150       #Maxvalue for FWHM
g1_fwhm_min = 1         #Minvalue for FWHM

g1_h = 15           #Guess for peak Intensity
g1_h_max = 40      #Maxvalue for peak Intesity
g1_h_min = 0         #Minvalue for peak Intensity

bkg_offset = 650  #Background to be substracted

In [14]:
m = cl_sem.create_model()

#Background offset
bkg = hs.model.components1D.Offset()
#Gaussian peak
g1 = hs.model.components1D.Expression(
expression="height * exp(-(x - x0) ** 2 * 4 * log(2)/ fwhm ** 2)",
name="Perovskite Peak",
position="x0",
height=1,
fwhm=1,
x0=0,
module="numpy")
#Tweak guessed initial parameters
m.extend([g1, bkg])
g1.x0.value, g1.x0.bmax, g1.x0.bmin = g1_centre, g1_max, g1_min
g1.fwhm.value, g1.fwhm.bmax, g1.fwhm.bmin = g1_fwhm, g1_fwhm_max, g1_fwhm_min
g1.height.value, g1.height.bmax, g1.height.bmin = g1_h, g1_h_max, g1_h_min
bkg.offset.value = bkg_offset

In [15]:
#Fit for all the positions
m.multifit(bounded=True, show_progressbar=True)

C:\Users\jf631\Documents\GitHub\jordiferrero\hyperspy\hyperspy\model.py:1328: VisibleDeprecationWarning: The `iterpath` default will change from `'flyback'` to `'serpentine'`in HyperSpy version 2.0. Change `iterpath` to other than None to suppress thiswarning.
  warnings.warn(msg, VisibleDeprecationWarning)


In [16]:
m.print_current_values()

Parameter Name,Free,Value,Std,Min,Max
fwhm,True,8.71711,4.5814,1,150
height,True,1.16194,0.526664,,40
x0,True,522.988,1.93342,493,533
Parameter Name,Free,Value,Std,Min,Max
offset,True,-0.14509,0.0481327,,


In [17]:
#Plot the fit on the raw data
m.plot(plot_components=True)

In [18]:
m_x0 = g1.x0.as_signal()
m_x0.plot(cmap='inferno')
m_intensity = g1.height.as_signal()
m_intensity.plot(cmap='viridis')

### Particle segmentation
You can do particle segmentation using model fitting:

In [22]:
#Make mask to remove region where the intensity is below the mean value:
mask_treshold = m_intensity.data.mean()
mask = m_intensity.data > mask_treshold #Returns a boolean matrix mask
plt.imshow(mask)


## END
